In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler


In [2]:
df = pd.read_csv("cleaned_retail_data.csv")
df.head()


,date,store_type,product_category,region,brand,sales,inventory,price,discount
0,2023-01-01,Urban,Electronics,North,Brand_A,120,30,15000,0.10
1,2023-01-02,Urban,Grocery,North,Brand_B,300,200,150,0.05
2,2023-01-03,Rural,Electronics,South,Brand_A,90,25,15000,0.15
3,2023-01-04,Rural,Clothing,South,Brand_C,200,60,1200,0.20
4,2023-01-05,Urban,Grocery,East,Brand_B,350,220,150,0.00


In [4]:
df['date'] = pd.to_datetime(df['date'])


In [6]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['week'] = df['date'].dt.isocalendar().week.astype(int)
df['weekday'] = df['date'].dt.dayofweek
df['quarter'] = df['date'].dt.quarter


In [7]:
df['is_weekend'] = np.where(df['weekday'] >= 5, 1, 0)


In [8]:
def get_season(month):
    if month in [12,1,2]:
        return "Winter"
    elif month in [3,4,5]:
        return "Summer"
    elif month in [6,7,8]:
        return "Monsoon"
    else:
        return "Autumn"

df['season'] = df['month'].apply(get_season)


In [9]:
le = LabelEncoder()

df['store_type_enc'] = le.fit_transform(df['store_type'])
df['region_enc'] = le.fit_transform(df['region'])
df['brand_enc'] = le.fit_transform(df['brand'])


In [10]:
df = pd.get_dummies(
    df,
    columns=['product_category','season'],
    drop_first=True
)


In [11]:
df = df.sort_values('date')

df['rolling_7day_sales'] = df['sales'].rolling(7, min_periods=1).mean()
df['rolling_30day_sales'] = df['sales'].rolling(30, min_periods=1).mean()


In [12]:
df['discount_impact'] = df['sales'] * df['discount']


In [13]:
df['price_elasticity'] = df['sales'] / df['price']


In [14]:
scaler = StandardScaler()

num_cols = [
    'sales','inventory','price','discount',
    'rolling_7day_sales','rolling_30day_sales',
    'discount_impact','price_elasticity'
]

df[num_cols] = scaler.fit_transform(df[num_cols])


In [15]:
corr_matrix = df[num_cols].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]
to_drop


['inventory', 'price', 'rolling_30day_sales', 'price_elasticity']

In [16]:
df.drop(columns=to_drop, inplace=True)


In [17]:
df.head()

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   date                          12 non-null     datetime64[ns]
 1   store_type                    12 non-null     object        
 2   region                        12 non-null     object        
 3   brand                         12 non-null     object        
 4   sales                         12 non-null     float64       
 5   discount                      12 non-null     float64       
 6   year                          12 non-null     int32         
 7   month                         12 non-null     int32         
 8   week                          12 non-null     int64         
 9   weekday                       12 non-null     int32         
 10  quarter                       12 non-null     int32         
 11  is_weekend                    12 n

In [18]:
df.to_csv("engineered_retail_data.csv", index=False)
